In [46]:
#ms = 20220410T0130  
#dt = datetime.fromtimestamp(ms/1000.0)
#dt

In [47]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from statistics import mean

api_key = "3U8Z3613Y27KCPIF"
url_interest = 'https://www.alphavantage.co/query?function=FEDERAL_FUNDS_RATE&interval=monthlytopics&apikey=3U8Z3613Y27KCPIF'
url_CPI = 'https://www.alphavantage.co/query?function=CPI&interval=monthly&apikey=3U8Z3613Y27KCPIF'
r1 = requests.get(url_interest)
r2 = requests.get(url_CPI)
data1 = r1.json()
data2 = r2.json()


print(data1)
print(data2)

{'name': 'Effective Federal Funds Rate', 'interval': 'monthly', 'unit': 'percent', 'data': [{'date': '2025-02-01', 'value': '4.33'}, {'date': '2025-01-01', 'value': '4.33'}, {'date': '2024-12-01', 'value': '4.48'}, {'date': '2024-11-01', 'value': '4.64'}, {'date': '2024-10-01', 'value': '4.83'}, {'date': '2024-09-01', 'value': '5.13'}, {'date': '2024-08-01', 'value': '5.33'}, {'date': '2024-07-01', 'value': '5.33'}, {'date': '2024-06-01', 'value': '5.33'}, {'date': '2024-05-01', 'value': '5.33'}, {'date': '2024-04-01', 'value': '5.33'}, {'date': '2024-03-01', 'value': '5.33'}, {'date': '2024-02-01', 'value': '5.33'}, {'date': '2024-01-01', 'value': '5.33'}, {'date': '2023-12-01', 'value': '5.33'}, {'date': '2023-11-01', 'value': '5.33'}, {'date': '2023-10-01', 'value': '5.33'}, {'date': '2023-09-01', 'value': '5.33'}, {'date': '2023-08-01', 'value': '5.33'}, {'date': '2023-07-01', 'value': '5.12'}, {'date': '2023-06-01', 'value': '5.08'}, {'date': '2023-05-01', 'value': '5.06'}, {'date

In [48]:
df = pd.json_normalize(data1)
df

,name,interval,unit,data
0,Effective Federal Funds Rate,monthly,percent,"[{'date': '2025-02-01', 'value': '4.33'}, {'da..."


In [49]:

data1_table = pd.DataFrame(data1)
data2_table = pd.DataFrame(data2)

In [50]:
values = [item["value"] for item in data1["data"]]
dates = [item["date"] for item in data1["data"]]
print(dates)
print(values)

['2025-02-01', '2025-01-01', '2024-12-01', '2024-11-01', '2024-10-01', '2024-09-01', '2024-08-01', '2024-07-01', '2024-06-01', '2024-05-01', '2024-04-01', '2024-03-01', '2024-02-01', '2024-01-01', '2023-12-01', '2023-11-01', '2023-10-01', '2023-09-01', '2023-08-01', '2023-07-01', '2023-06-01', '2023-05-01', '2023-04-01', '2023-03-01', '2023-02-01', '2023-01-01', '2022-12-01', '2022-11-01', '2022-10-01', '2022-09-01', '2022-08-01', '2022-07-01', '2022-06-01', '2022-05-01', '2022-04-01', '2022-03-01', '2022-02-01', '2022-01-01', '2021-12-01', '2021-11-01', '2021-10-01', '2021-09-01', '2021-08-01', '2021-07-01', '2021-06-01', '2021-05-01', '2021-04-01', '2021-03-01', '2021-02-01', '2021-01-01', '2020-12-01', '2020-11-01', '2020-10-01', '2020-09-01', '2020-08-01', '2020-07-01', '2020-06-01', '2020-05-01', '2020-04-01', '2020-03-01', '2020-02-01', '2020-01-01', '2019-12-01', '2019-11-01', '2019-10-01', '2019-09-01', '2019-08-01', '2019-07-01', '2019-06-01', '2019-05-01', '2019-04-01', '2019

In [51]:
data1_table

,name,interval,unit,data
0,Effective Federal Funds Rate,monthly,percent,"{'date': '2025-02-01', 'value': '4.33'}"
1,Effective Federal Funds Rate,monthly,percent,"{'date': '2025-01-01', 'value': '4.33'}"
2,Effective Federal Funds Rate,monthly,percent,"{'date': '2024-12-01', 'value': '4.48'}"
3,Effective Federal Funds Rate,monthly,percent,"{'date': '2024-11-01', 'value': '4.64'}"
4,Effective Federal Funds Rate,monthly,percent,"{'date': '2024-10-01', 'value': '4.83'}"
...,...,...,...,...
843,Effective Federal Funds Rate,monthly,percent,"{'date': '1954-11-01', 'value': '0.83'}"
844,Effective Federal Funds Rate,monthly,percent,"{'date': '1954-10-01', 'value': '0.85'}"
845,Effective Federal Funds Rate,monthly,percent,"{'date': '1954-09-01', 'value': '1.07'}"
846,Effective Federal Funds Rate,monthly,percent,"{'date': '1954-08-01', 'value': '1.22'}"


In [52]:
data1_table["values"] = values
data1_table["dates"] = dates
data1_table

,name,interval,unit,data,values,dates
0,Effective Federal Funds Rate,monthly,percent,"{'date': '2025-02-01', 'value': '4.33'}",4.33,2025-02-01
1,Effective Federal Funds Rate,monthly,percent,"{'date': '2025-01-01', 'value': '4.33'}",4.33,2025-01-01
2,Effective Federal Funds Rate,monthly,percent,"{'date': '2024-12-01', 'value': '4.48'}",4.48,2024-12-01
3,Effective Federal Funds Rate,monthly,percent,"{'date': '2024-11-01', 'value': '4.64'}",4.64,2024-11-01
4,Effective Federal Funds Rate,monthly,percent,"{'date': '2024-10-01', 'value': '4.83'}",4.83,2024-10-01
...,...,...,...,...,...,...
843,Effective Federal Funds Rate,monthly,percent,"{'date': '1954-11-01', 'value': '0.83'}",0.83,1954-11-01
844,Effective Federal Funds Rate,monthly,percent,"{'date': '1954-10-01', 'value': '0.85'}",0.85,1954-10-01
845,Effective Federal Funds Rate,monthly,percent,"{'date': '1954-09-01', 'value': '1.07'}",1.07,1954-09-01
846,Effective Federal Funds Rate,monthly,percent,"{'date': '1954-08-01', 'value': '1.22'}",1.22,1954-08-01


In [53]:
del data1_table["data"]

In [54]:
data1_table

,name,interval,unit,values,dates
0,Effective Federal Funds Rate,monthly,percent,4.33,2025-02-01
1,Effective Federal Funds Rate,monthly,percent,4.33,2025-01-01
2,Effective Federal Funds Rate,monthly,percent,4.48,2024-12-01
3,Effective Federal Funds Rate,monthly,percent,4.64,2024-11-01
4,Effective Federal Funds Rate,monthly,percent,4.83,2024-10-01
...,...,...,...,...,...
843,Effective Federal Funds Rate,monthly,percent,0.83,1954-11-01
844,Effective Federal Funds Rate,monthly,percent,0.85,1954-10-01
845,Effective Federal Funds Rate,monthly,percent,1.07,1954-09-01
846,Effective Federal Funds Rate,monthly,percent,1.22,1954-08-01


In [55]:
data2_table

,name,interval,unit,data
0,Consumer Price Index for all Urban Consumers,monthly,index 1982-1984=100,"{'date': '2025-01-01', 'value': '317.671'}"
1,Consumer Price Index for all Urban Consumers,monthly,index 1982-1984=100,"{'date': '2024-12-01', 'value': '315.605'}"
2,Consumer Price Index for all Urban Consumers,monthly,index 1982-1984=100,"{'date': '2024-11-01', 'value': '315.493'}"
3,Consumer Price Index for all Urban Consumers,monthly,index 1982-1984=100,"{'date': '2024-10-01', 'value': '315.664'}"
4,Consumer Price Index for all Urban Consumers,monthly,index 1982-1984=100,"{'date': '2024-09-01', 'value': '315.301'}"
...,...,...,...,...
1340,Consumer Price Index for all Urban Consumers,monthly,index 1982-1984=100,"{'date': '1913-05-01', 'value': '9.700'}"
1341,Consumer Price Index for all Urban Consumers,monthly,index 1982-1984=100,"{'date': '1913-04-01', 'value': '9.800'}"
1342,Consumer Price Index for all Urban Consumers,monthly,index 1982-1984=100,"{'date': '1913-03-01', 'value': '9.800'}"
1343,Consumer Price Index for all Urban Consumers,monthly,index 1982-1984=100,"{'date': '1913-02-01', 'value': '9.800'}"


In [56]:
url_k = "./btc.csv"
df_kaggle = pd.read_csv(url_k)
print (df_kaggle)


            Date     Open     High      Low    Close        Volume
0     2010-07-18      0.0      0.1      0.1      0.1  8.000000e+01
1     2010-07-19      0.1      0.1      0.1      0.1  5.700000e+02
2     2010-07-20      0.1      0.1      0.1      0.1  2.600000e+02
3     2010-07-21      0.1      0.1      0.1      0.1  5.800000e+02
4     2010-07-22      0.1      0.1      0.1      0.1  2.160000e+03
...          ...      ...      ...      ...      ...           ...
4989  2024-03-15  71387.9  72357.1  65630.7  69403.8  7.832045e+10
4990  2024-03-16  69392.5  70046.3  64801.4  65315.1  4.684220e+10
4991  2024-03-17  65316.3  68845.7  64545.3  68390.6  4.471686e+10
4992  2024-03-18  68371.3  68897.1  66594.2  67548.6  4.926158e+10
4993  2024-03-19  67556.1  68106.9  61536.2  61912.8  7.421584e+10

[4994 rows x 6 columns]


In [57]:
data3_table = pd.DataFrame(df_kaggle)

In [58]:
data3_table

,Date,Open,High,Low,Close,Volume
0,2010-07-18,0.0,0.1,0.1,0.1,8.000000e+01
1,2010-07-19,0.1,0.1,0.1,0.1,5.700000e+02
2,2010-07-20,0.1,0.1,0.1,0.1,2.600000e+02
3,2010-07-21,0.1,0.1,0.1,0.1,5.800000e+02
4,2010-07-22,0.1,0.1,0.1,0.1,2.160000e+03
...,...,...,...,...,...,...
4989,2024-03-15,71387.9,72357.1,65630.7,69403.8,7.832045e+10
4990,2024-03-16,69392.5,70046.3,64801.4,65315.1,4.684220e+10
4991,2024-03-17,65316.3,68845.7,64545.3,68390.6,4.471686e+10
4992,2024-03-18,68371.3,68897.1,66594.2,67548.6,4.926158e+10
